In [1]:
import torch
import torch.nn as nn  # 神经网络模块，如线性层 (Linear)、嵌入层 (Embedding) 等
import torch.nn.functional as F  # 操作的函数式接口，如交叉熵 (cross_entropy)、SiLU 激活函数等
from torch.utils.data import Dataset, DataLoader  # 加载数据集的基础类和实用工具
from torch.cuda.amp import autocast, GradScaler  # 自动混合精度 (AMP) 工具，用于实现更快、更低显存占用的训练

import math  # 标准数学运算（例如平方根 sqrt、指数 exp、余弦 cos）
import random  # Python 的随机数生成工具（用于设置随机种子）
import numpy as np  # 数值计算库，用于随机种子设置和通用的数组操作

from datasets import load_dataset  # Hugging Face 的数据集库，用于流式加载大规模数据集
from tqdm import tqdm  # 进度条可视化库，非常适合循环过程

import time  # 时间工具，用于测量时长
from transformers import AutoTokenizer  # 仅需一行代码即可从 HuggingFace 加载预训练的分词器 (Tokenizer)

from dataclasses import dataclass  # 定义简单的配置类 (Configs)，减少冗余代码
from typing import List, Optional  # 类型提示 (Type hints)，提高代码可读性和工具支持

import warnings  # 抑制或处理警告信息
import os  # 文件系统操作
import pickle  # Python 对象序列化工具（用于保存/加载预处理后的数据集）

warnings.filterwarnings('ignore')  # 忽略警告，使训练过程中的输出更加整洁

In [2]:
# Utility Functions

def set_seed(seed: int = 42):
    """Set all random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Set all seed to {seed}")

In [3]:
# Model Configuration
@dataclass
class ModelConfig:
    # Model architecture
    d_model: int= 384
    n_heads: int = 8
    n_layers: int = 6
    d_ff: int = 1536
    batch_size: int = 24
    max_steps: int = 500

    # Qwen3-like parameters
    n_kv_heads: int = 4
    sliding_window: int = 4096
    attention_bias: bool = False
    rms_norm_eps: float = 1e-6

    #Training parameters
    gradient_accumulation_steps: int = 4
    muon_lr: float = 0.01

    # Data parameters
    max_seq_len: int = 256
    num_documents: int = 2000
    max_tokens: int = 500000

    #Evaluation
    eval_every: int = 100
    eval_steps: int = 50

    # Regularization
    weight_decay: float = 0.1
    dropout: float = 0.1
    grad_clip: float = 1.0

    #Technical
    use_amp: bool = True
    vocab_size: Optional[int] = None

    def __post_init__(self):
        self.d_k = self.d_model // self.n_heads
        assert self.d_model % self.n_heads == 0, "d_model must be divisible by n_heads"
        assert self.n_heads % self.n_kv_heads == 0, "n_heads must must be divisible by n_kv_heads"
        self.n_kv_groups = self.n_heads // self.n_kv_heads



In [4]:
# GQA helper

def repeat_kv(hidden_state: torch.Tensor, n_rep: int)-> torch.Tensor:
    """
    (batch, num_kv_heads, seqLen, head_dim) -> (bath, n_heads, seqLen, head_dim)
    """ 

    batch, num_key_value_heads, slen, head_dim = hidden_state.shape
    if n_rep == 1:
        return hidden_state
    hidden_state = hidden_state[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    return hidden_state.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)

In [5]:
import torch

def zeropower_via_newtonschulz5(G: torch.Tensor, steps: int = 5) -> torch.Tensor:
    """
    通过 Newton-Schulz 迭代计算矩阵的零次幂，实现 G 的标准正交化（Orthogonalization）。
    该函数会将输入矩阵 G 转换为一个行与行（或列与列）之间互相垂直且长度为 1 的矩阵。
    """
    assert G.ndim >= 2
    # a, b, c 是 5 阶 Newton-Schulz 迭代的特定系数，用于快速逼近正交矩阵
    a, b, c = (3.4445, -4.7750, 2.0315)
    X = G.bfloat16()

    # 如果行数大于列数，先进行转置，确保对较小的维度进行计算以提高效率
    if G.size(-2) > G.size(-1):
        X = X.mT

    # 初始归一化：除以 Frobenius 范数，将矩阵缩放到迭代收敛范围内，防止数值爆炸
    X = X / (X.norm(dim=(-2, -1), keepdim=True) + 1e-7)

    # Newton-Schulz 核心循环
    for _ in range(steps):
        A = X @ X.mT          # 计算 X 乘以自身的转置，衡量其与单位矩阵的距离
        B = b * A + c * A @ A # 计算多项式的二阶和四阶项
        X = a * X + B @ X     # 更新 X，使其进一步逼近标准正交矩阵

    # 如果之前为了计算方便转置了矩阵，现在将其转置回原始形状
    if G.size(-2) > G.size(-1):
        X = X.mT

    return X

class Muon(torch.optim.Optimizer):
    """
    Muon 优化器 - 通过 Newton-Schulz 迭代实现动量正交化 (MomentUm Orthogonalized by Newton-schulz)
    
    该优化器特别适用于 Transformer 的线性层，通过强制更新方向的正交性来加速收敛。
    """
    def __init__(self, params, lr=0.02, momentum=0.95, nesterov=True, ns_steps=5):
        # 默认超参数配置：学习率、动量、是否使用 Nesterov 动量、Newton-Schulz 迭代步数
        defaults = dict(lr=lr, momentum=momentum, nesterov=nesterov, ns_steps=ns_steps)
        super().__init__(params, defaults)

    @torch.no_grad()
    def step(self):
        """执行单一的优化步（参数更新）"""
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None:
                    continue

                g = p.grad
                state = self.state[p]

                # 第一次运行周期：初始化动量缓存（momentum buffer）
                if "momentum_buffer" not in state:
                    state["momentum_buffer"] = torch.zeros_like(g)

                buf = state["momentum_buffer"]
                
                # 更新动量缓存：buf = momentum * buf + (1 - momentum) * grad
                # 使用 .lerp_ (线性插值) 是为了在 GPU 上获得更高的计算效率
                buf.lerp_(g, 1 - group["momentum"])
                
                # 如果启用 Nesterov，则结合当前梯度和动量；否则直接使用标准动量
                g = g.lerp_(buf, group["momentum"]) if group["nesterov"] else buf
                
                # 核心步骤：通过 Newton-Schulz 迭代将梯度转换为（接近）标准正交矩阵
                # 这保证了权重的更新方向在每一维上都是独立且等效的
                g = zeropower_via_newtonschulz5(g, steps=group["ns_steps"])
                
                # 更新参数权重：
                # 这里应用了基于参数张量长宽比（aspect ratio）的自适应缩放。
                # 对于高大于宽的矩阵（height > width），学习率会乘上 √(height/width) 得到增强。
                scale = max(1, p.size(-2) / p.size(-1))**0.5
                p.add_(g.view_as(p), alpha=-group["lr"] * scale)

In [6]:
import os
import pickle
from tqdm import tqdm
from transformers import AutoTokenizer
from datasets import load_dataset

# 数据加载与缓存函数
def load_and_cache_data(config: ModelConfig, cache_dir: str = "data_cache"):
    """加载并缓存分词后的数据，以避免重复处理"""
    
    # 确保缓存目录存在，如果不存在则创建一个
    os.makedirs(cache_dir, exist_ok=True)
    
    # 根据配置中的文档数量和最大 Token 数生成一个唯一的文件名
    cache_file = f"{cache_dir}/tokenized_data_{config.num_documents}_{config.max_tokens}.pkl"

    # --- 步骤 1：检查缓存 ---
    if os.path.exists(cache_file):
        print(f"📦 正在从缓存中加载数据: {cache_file}")
        with open(cache_file, 'rb') as f:
            cached_data = pickle.load(f)

        # 从缓存中提取数据
        texts = cached_data['texts']
        tokenizer = cached_data['tokenizer']
        tokens = cached_data['tokens']
        # 将分词器的词表大小同步给配置对象
        config.vocab_size = tokenizer.vocab_size

        print(f"✅ 加载成功：包含 {len(texts)} 篇文档，共 {len(tokens):,} 个 Token")
        return texts, tokenizer, tokens

    # --- 步骤 2：处理新数据（如果没有缓存） ---
    print(f"🔄 正在处理新数据（处理后将自动缓存以备后用）")

    # 1. 加载分词器 (使用 SmolLM 预训练分词器)
    tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM-135M")
    # 如果没有定义填充符（Pad Token），则使用终止符（EOS Token）代替
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # 2. 从 HuggingFace 加载数据集 (使用流式加载 streaming=True，节省显存)
    dataset = load_dataset("HuggingFaceTB/smollm-corpus", "cosmopedia-v2", split="train", streaming=True)

    texts = []
    # 遍历数据集，只提取配置中指定的文档数量
    for i, item in enumerate(dataset):
        if i >= config.num_documents:
            break
        # 每篇文档截取前 3000 个字符，防止单条数据过大
        texts.append(item["text"][:3000])

    print(f"已加载 {len(texts)} 篇原始文档")

    # 3. 执行分词操作
    print("正在进行分词...")
    all_tokens = []
    # 使用 tqdm 显示分词进度条
    for text in tqdm(texts, desc="分词进度"):
        tokens = tokenizer.encode(text, add_special_tokens=False)
        all_tokens.extend(tokens)

    # 4. 截断到配置要求的最大 Token 数量
    tokens = all_tokens[:config.max_tokens]
    print(f"最终使用 Token 数量: {len(tokens):,}")
    config.vocab_size = tokenizer.vocab_size

    # --- 步骤 3：保存到缓存 ---
    cached_data = {'texts': texts, 'tokenizer': tokenizer, 'tokens': tokens}
    with open(cache_file, 'wb') as f:
        pickle.dump(cached_data, f)

    print(f"💾 数据已成功缓存至: {cache_file}")
    return texts, tokenizer, tokens

In [7]:
class TextTokenDataset(Dataset):
    def __init__(self, tokens: List[int], seq_len: int = 512):
        """
        初始化数据集
        :param tokens: 整个语料库分词后的 Token 列表（一个巨大的整数列表）
        :param seq_len: 每个训练样本的序列长度（窗口大小）
        """
        self.tokens = tokens
        self.seq_len = seq_len

    def __len__(self):
        """
        返回数据集中样本的总数
        计算方式：总 Token 数减去序列长度。
        例如：有 100 个 token，窗口设为 10，则可以滑动出 90 个不同的起点。
        使用 max(0, ...) 是为了防止 tokens 长度小于 seq_len 时出现负数导致报错。
        """
        return max(0, len(self.tokens) - self.seq_len)

    def __getitem__(self, idx):
        """
        根据给定的索引 idx 获取一个训练样本 (x, y)
        x: 输入序列，从 idx 开始，长度为 seq_len
        y: 标签序列（Target），是 x 往后挪动一位的结果
        """
        # 提取输入序列 x: [tokens[idx], ..., tokens[idx + seq_len - 1]]
        x = torch.tensor(self.tokens[idx : idx + self.seq_len], dtype=torch.long)
        
        # 提取目标序列 y: [tokens[idx + 1], ..., tokens[idx + seq_len]]
        # 这就是所谓的“下一个词预测”任务，每一个位置的 x 对应的 y 都是它的下一个字符
        y = torch.tensor(self.tokens[idx + 1 : idx + self.seq_len + 1], dtype=torch.long)
        
        return x, y

In [8]:
class Rotary(nn.Module):
    def __init__(self, dim: int, max_seq_len: int):
        """
        初始化 RoPE 旋转位置编码
        :param dim: 每个注意力头的维度 (head_dim)
        :param max_seq_len: 支持的最大序列长度
        """
        super().__init__()
        
        # --- 步骤 1: 计算角频率 (Angular Frequencies) ---
        # 按照公式计算频率：theta_i = 10000^(-2i/dim)
        # 这里 dim//4 是因为 RoPE 通常只作用于前一半维度，且每两个维度组成一个旋转复数平面
        angular_freq = (1 / 10000) ** torch.linspace(0, 1, steps=dim//4, dtype=torch.float32)
        
        # 这里将计算出的频率与一组 0 拼接，意味着只有前半部分维度会进行旋转
        # 这种做法常见于某些高效实现，可以保留一部分维度的绝对位置信息
        angular_freq = torch.cat([angular_freq, angular_freq.new_zeros(dim//4)])
        
        # --- 步骤 2: 构建位置矩阵 ---
        # t 是位置索引 [0, 1, 2, ..., max_seq_len-1]
        t = torch.arange(max_seq_len, dtype=torch.float32)
        
        # 使用外积计算所有位置的 theta 值: theta = t * angular_freq
        # 结果维度为 (max_seq_len, dim//2)
        theta = torch.einsum("i,j -> ij", t, angular_freq)
        
        # 预先计算并缓存 cos 和 sin 值，提高推理效率
        # persistent=False 表示这些缓存不会被保存在模型的 state_dict 中
        self.register_buffer('cos', theta.cos(), persistent=False)
        self.register_buffer('sin', theta.sin(), persistent=False)

    def forward(self, x_BTHD: torch.Tensor):
        """
        执行旋转变换
        :param x_BTHD: 输入张量，形状为 (Batch, Time/SeqLen, Heads, head_dim)
        """
        # 检查缓存的 cos/sin 长度是否足够覆盖当前输入序列
        assert self.cos.size(0) >= x_BTHD.size(-3)
        
        # 提取当前序列长度对应的 cos 和 sin，并增加维度以便进行广播计算
        # 形状变为 (1, SeqLen, 1, head_dim//2)
        cos, sin = self.cos[None, :x_BTHD.size(-3), None, :], self.sin[None, :x_BTHD.size(-3), None, :]
        
        # 将输入张量在最后一个维度（head_dim）切分为两半
        x_BTHD_float = x_BTHD.to(dtype=torch.float32)
        x1, x2 = x_BTHD_float.chunk(2, dim=-1)
        
        # --- 步骤 3: 应用旋转矩阵变换 ---
        # 这是复数乘法的实数等价形式（旋转变换公式）：
        # y1 = x1 * cos + x2 * sin
        # y2 = x2 * cos - x1 * sin
        y1 = x1 * cos + x2 * sin
        y2 = x1 * (-sin) + x2 * cos
        
        # 将处理后的两部分重新拼接，并转回原始数据类型（如 bfloat16）
        return torch.cat((y1, y2), dim=-1).type_as(x_BTHD)

In [9]:
class Qwen3Attention(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.d_model = config.d_model
        self.n_heads = config.n_heads
        self.n_kv_heads = config.n_kv_heads
        self.n_kv_groups = config.n_kv_groups
        self.d_k = config.d_k

        # 为 Q, K, V 分别设置线性投影层
        self.q_proj = nn.Linear(self.d_model, self.n_heads * self.d_k, bias=config.attention_bias)
        self.k_proj = nn.Linear(self.d_model, self.n_kv_heads * self.d_k, bias=config.attention_bias)
        self.v_proj = nn.Linear(self.d_model, self.n_kv_heads * self.d_k, bias=config.attention_bias)
        self.w_o = nn.Linear(self.d_model, self.d_model, bias=False)

        # QK-Normalization 层：用于提高超大规模模型训练的稳定性
        # 这种做法可以防止点积注意力的值在训练初期溢出
        self.q_norm = nn.RMSNorm(self.d_k, eps=config.rms_norm_eps)
        self.k_norm = nn.RMSNorm(self.d_k, eps=config.rms_norm_eps)

        # 旋转位置编码 (RoPE)
        self.rotary = Rotary(self.d_k, config.max_seq_len)
        self.dropout = config.dropout

    def forward(self, x):
        batch_size, seq_len = x.size(0), x.size(1)

        # 1. 执行线性投影，生成 Q, K, V 向量
        q = self.q_proj(x)
        k = self.k_proj(x)
        v = self.v_proj(x)

        # 2. 将输出重塑（Reshape）为多头结构
        # 形状变化: (batch, seq_len, d_model) -> (batch, seq_len, n_heads, d_k)
        q = q.view(batch_size, seq_len, self.n_heads, self.d_k)
        k = k.view(batch_size, seq_len, self.n_kv_heads, self.d_k)
        v = v.view(batch_size, seq_len, self.n_kv_heads, self.d_k)

        # 3. 应用 QK-Norm
        # 在进入点积计算前对 Q 和 K 进行归一化
        q = self.q_norm(q)
        k = self.k_norm(k)

        # 4. 应用旋转位置编码 (RoPE)
        # 注意：这里进行了维度置换，以匹配 rotary 模块的输入要求
        # (batch, seq_len, n_heads, d_k) -> (batch, n_heads, seq_len, d_k)
        q = self.rotary(q.permute(0, 2, 1, 3)).permute(0, 2, 1, 3)
        k = self.rotary(k.permute(0, 2, 1, 3)).permute(0, 2, 1, 3)

        # 为注意力计算准备张量：将 head 维度移到第 1 维
        Q = q.transpose(1, 2)
        K = k.transpose(1, 2)
        V = v.transpose(1, 2)

        # 5. 为 GQA（分组查询注意力）重复 K 和 V 的头
        # 将 KV 头的数量增加到与 Q 头一致
        K = repeat_kv(K, self.n_kv_groups)
        V = repeat_kv(V, self.n_kv_groups)

        # 6. 缩放点积注意力 (Scaled Dot-Product Attention)
        # 使用 Flash Attention 优化（如果可用），同时应用因果掩码 (is_causal=True)
        attn_output = F.scaled_dot_product_attention(
            Q, K, V, is_causal=True, dropout_p=self.dropout if self.training else 0.0
        )

        # 7. 重塑形状并进行最后的线性输出变换
        # 将多个头的结果合并回 (batch, seq_len, d_model)
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        return self.w_o(attn_output)

In [10]:

class SwiGLUFeedForward(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float = 0.1):
        """
        SwiGLU 前馈神经网络模块
        :param d_model: 模型的隐藏层维度
        :param d_ff: 中间层的维度（通常设为 d_model 的 2.7 到 4 倍）
        :param dropout: 丢弃率
        """
        super().__init__()
        # 门控投影层 (Gate Projection)：负责决定哪些信息可以通过
        self.gate_proj = nn.Linear(d_model, d_ff, bias=False)
        
        # 维度还原投影层 (Down Projection)：将维度从 d_ff 还原回 d_model
        self.down_proj = nn.Linear(d_ff, d_model, bias=False)
        
        # 上采样投影层 (Up Projection)：负责提取更高维度的特征
        self.up_proj = nn.Linear(d_model, d_ff, bias=False)
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # SwiGLU 激活函数的具体实现逻辑：
        # 1. 对 gate_proj 的输出应用 SiLU (即 Swish) 激活函数
        # 2. 将激活后的结果与 up_proj 的输出进行逐元素相乘（Hadamard Product）
        # 3. 这种“门控”机制能让模型更灵活地过滤特征
        activated_x = F.silu(self.gate_proj(x)) * self.up_proj(x)
        
        # 最后进行线性投影并应用 Dropout 增强泛化能力
        return self.down_proj(self.dropout(activated_x))

In [11]:
class TransformerBlock(nn.Module):
    def __init__(self, config: ModelConfig):  # Pass the entire config object
        super().__init__()
        self.attention = Qwen3Attention(config)
        self.feed_forward = SwiGLUFeedForward(config.d_model, config.d_ff, config.dropout)
        self.norm1 = nn.RMSNorm(config.d_model, eps=config.rms_norm_eps)
        self.norm2 = nn.RMSNorm(config.d_model, eps=config.rms_norm_eps)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        attn_out = self.attention(self.norm1(x))
        x = x + self.dropout(attn_out)
        ff_out = self.feed_forward(self.norm2(x))
        x = x + self.dropout(ff_out)
        return x


In [12]:
class MinimalLLM(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.config = config

        self.token_embedding = nn.Embedding(config.vocab_size, config.d_model)
        self.position_dropout = nn.Dropout(config.dropout)

        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(config) for _ in range(config.n_layers)
        ])

        self.norm = nn.RMSNorm(config.d_model, eps=config.rms_norm_eps)
        self.output_dropout = nn.Dropout(config.dropout)

        # Tie weights
        # This ties the output layer (`lm_head`) weights to the input token embedding weights so the model shares parameters between input and output, reducing memory and improving generalization.
        self.lm_head = nn.Linear(config.d_model, config.vocab_size, bias=False)
        self.lm_head.weight = self.token_embedding.weight

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, x):
        x = self.token_embedding(x) * math.sqrt(self.config.d_model)
        x = self.position_dropout(x)

        for block in self.transformer_blocks:
            x = block(x)

        x = self.norm(x)
        x = self.output_dropout(x)
        logits = self.lm_head(x)
        return logits


In [13]:

def evaluate_model(model: nn.Module, val_loader: DataLoader, config: ModelConfig):
    """评估模型性能"""
    # 将模型设置为评估模式：
    # 这会关闭 Dropout 和 BatchNorm 的随机性，确保预测结果是稳定的
    model.eval()
    
    total_loss = 0
    total_tokens = 0
    total_correct = 0

    # 获取模型所在的设备（CPU 或 GPU）
    device = next(model.parameters()).device

    # 禁用梯度计算：
    # 评估时不需要计算梯度（不需要反向传播），这样可以大幅节省显存并加快计算速度
    with torch.no_grad():
        for i, (x, y) in enumerate(val_loader):
            # 如果达到了配置中指定的评估步数，则停止，以限制评估时间
            if i >= config.eval_steps:
                break

            # 将输入序列 (x) 和目标序列 (y) 移动到 GPU/设备上
            x, y = x.to(device), y.to(device)

            # 如果启用了混合精度训练 (AMP)，这里也会同步使用
            # 这可以在保证精度的前提下提高推理速度
            with autocast(enabled=config.use_amp):
                # 前向传播：获取模型对输入序列的预测结果（Logits）
                logits = model(x)

                # 计算交叉熵损失（Cross-Entropy Loss）：
                # 需要将形状展平：(batch_size * seq_len, vocab_size) 和 (batch_size * seq_len)
                # 这样才能对所有位置的 Token 进行统一的损失计算
                loss = F.cross_entropy(logits.view(-1, config.vocab_size), y.view(-1))

            # 累加总损失：使用 loss.item() 乘以当前 Batch 的 Token 总数
            total_loss += loss.item() * y.numel()
            # 记录处理过的 Token 总数
            total_tokens += y.numel()

            # 获取预测结果：在词表维度（最后一维）取最大值的索引（Argmax）
            predictions = logits.argmax(dim=-1)
            # 统计预测正确的 Token 数量，用于计算准确率
            total_correct += (predictions == y).sum().item()

    # 计算平均损失和准确率
    avg_loss = total_loss / total_tokens
    accuracy = total_correct / total_tokens
    
    # 计算困惑度 (Perplexity)：它是 Loss 的指数形式，代表模型预测下一个词时的“迷茫程度”
    # 限制最大值为 exp(20) 以防止溢出
    perplexity = math.exp(min(avg_loss, 20))

    # 考试结束，将模型重新切回训练模式，准备接下来的训练
    model.train()
    
    return {
        'val_loss': avg_loss, 
        'val_accuracy': accuracy, 
        'val_perplexity': perplexity
    }

In [14]:
def setup_muon_optimizer(model: nn.Module, config: ModelConfig):
    """Setup Muon optimizer with hybrid approach"""
    muon_params = []
    adamw_params = []

    for name, param in model.named_parameters():
        if (param.ndim == 2 and
            'token_embedding' not in name and
            'norm' not in name and
            param.requires_grad):
            muon_params.append(param)
        else:
            adamw_params.append(param)

    print(f"  Muon parameters: {sum(p.numel() for p in muon_params):,}")
    print(f"  AdamW parameters: {sum(p.numel() for p in adamw_params):,}")

    muon_optimizer = Muon(muon_params, lr=config.muon_lr, momentum=0.95)
    adamw_optimizer = torch.optim.AdamW(adamw_params, lr=config.muon_lr*0.1, weight_decay=config.weight_decay)

    return [muon_optimizer, adamw_optimizer]



In [15]:
def train_model(config: ModelConfig, train_loader: DataLoader, val_loader: DataLoader):
    """使用 Muon 优化器训练模型"""
    print(f"\n🚀 开始训练 (架构: Modern LLM, 优化器: Muon)")

    # 1. 初始化模型与环境
    set_seed(42) # 固定随机种子，确保实验可复现
    model = MinimalLLM(config)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    total_params = sum(p.numel() for p in model.parameters())
    print(f"  📊 总参数量: {total_params:,}")

    # 2. 设置优化器：这里会返回 [muon_optimizer, adamw_optimizer] 两个对象
    optimizers = setup_muon_optimizer(model, config)

    # 3. 设置学习率调度器 (Learning Rate Schedule)
    # 使用 Cosine Decay (余弦退火) 策略，并带有 Warmup (预热) 阶段
    schedulers = []
    for optimizer in optimizers:
        warmup_steps = config.max_steps // 20 # 前 5% 的步数用于预热
        
        def lr_lambda(step):
            # 预热阶段：学习率从 0 线性增加到设定值
            if step < warmup_steps:
                return step / warmup_steps
            # 退火阶段：学习率按余弦曲线下降至初始值的 10%
            else:
                progress = (step - warmup_steps) / (config.max_steps - warmup_steps)
                return 0.1 + 0.9 * 0.5 * (1 + math.cos(math.pi * progress))

            # 使用 LambdaLR 灵活控制学习率
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
        schedulers.append(scheduler)

    # 4. 准备混合精度训练 (AMP)
    # GradScaler 用于防止 FP16 训练时因梯度过小导致的数值下溢
    scaler = GradScaler() if config.use_amp else None

    # 5. 训练状态初始化
    model.train()
    step = 0
    start_time = time.time()
    best_val_loss = float('inf')

    # 使用 tqdm 显示进度条
    pbar = tqdm(total=config.max_steps, desc="正在训练")

    while step < config.max_steps:
        for batch_idx, (x, y) in enumerate(train_loader):
            if step >= config.max_steps:
                break

            x, y = x.to(device), y.to(device)

            # --- 前向传播与梯度累积 ---
            # 梯度累积允许我们在显存有限的情况下，模拟出更大的 Batch Size
            if config.use_amp:
                with autocast(): # 开启自动混合精度
                    logits = model(x)
                    loss = F.cross_entropy(logits.view(-1, config.vocab_size), y.view(-1))
                    # 将 Loss 除以累积步数，因为梯度是累加的
                    loss = loss / config.gradient_accumulation_steps
                scaler.scale(loss).backward() # 缩放 Loss 并反向传播
            else:
                logits = model(x)
                loss = F.cross_entropy(logits.view(-1, config.vocab_size), y.view(-1))
                loss = loss / config.gradient_accumulation_steps
                loss.backward()

            # --- 优化器更新步 (只有达到累积步数时才执行) ---
            if (step + 1) % config.gradient_accumulation_steps == 0:
                if config.use_amp:
                    # 在更新前先 unscale 梯度，以便进行梯度裁剪
                    for optimizer in optimizers:
                        scaler.unscale_(optimizer)
                    
                    # 梯度裁剪：防止梯度爆炸，维持训练稳定性
                    grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_clip)

                    # 分别更新两个优化器（Muon 和 AdamW）
                    for optimizer in optimizers:
                        scaler.step(optimizer)
                        optimizer.zero_grad() # 清空梯度，为下一轮做准备
                    for scheduler in schedulers:
                        scheduler.step() # 更新学习率
                    scaler.update() # 更新 Scaler 的缩放因子
                else:
                    grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_clip)
                    for optimizer in optimizers:
                        optimizer.step()
                        optimizer.zero_grad()
                    for scheduler in schedulers:
                        scheduler.step()

            # 6. 日志记录 (每 10 步记录一次)
            if step % 10 == 0:
                with torch.no_grad():
                    predictions = logits.argmax(dim=-1)
                    accuracy = (predictions == y).float().mean().item()
                    # 恢复真实的 Loss 值用于显示
                    current_loss = loss.item() * config.gradient_accumulation_steps
                    perplexity = math.exp(min(current_loss, 20))

                # 在进度条右侧实时更新关键指标
                pbar.set_postfix({
                    'loss': f'{current_loss:.4f}',
                    'acc': f'{accuracy:.3f}',
                    'ppl': f'{perplexity:.1f}',
                    'lr': f'{optimizers[0].param_groups[0]["lr"]:.2e}' # 显示 Muon 的学习率
                })

            # 7. 定期进行验证 (Evaluation)
            if step % config.eval_every == 0 and step > 0:
                eval_metrics = evaluate_model(model, val_loader, config)
                print(f"\n步数 {step}: 验证损失: {eval_metrics['val_loss']:.4f}, "
                      f"验证准确率: {eval_metrics['val_accuracy']:.4f}, "
                      f"验证困惑度: {eval_metrics['val_perplexity']:.2f}")

                # 如果验证损失破了纪录，则保存“最佳模型”
                if eval_metrics['val_loss'] < best_val_loss:
                    best_val_loss = eval_metrics['val_loss']
                    torch.save({
                        'model_state_dict': model.state_dict(),
                        'config': config,
                        'step': step,
                        'best_val_loss': best_val_loss,
                        'final_metrics': eval_metrics
                    }, 'best_model.pt')
                    print(f"💾 已保存最佳模型，当前验证损失: {best_val_loss:.4f}")

            step += 1
            if step % 10 == 0:
                pbar.update(10)

    pbar.close()

    # 8. 训练结束，保存最终模型
    training_time = time.time() - start_time
    print(f"  ⏱️ 训练完成，耗时 {training_time:.1f} 秒")

    final_eval = evaluate_model(model, val_loader, config)
    torch.save({
        'model_state_dict': model.state_dict(),
        'config': config,
        'step': step,
        'final_metrics': final_eval
    }, 'final_model.pt')
    print(f"💾 已保存最终模型至 final_model.pt")

    return model, final_eval

In [16]:
if __name__ == "__main__":
    # -----------------------------------------------------------------------------
    # 1. 系统环境检查
    # -----------------------------------------------------------------------------
    # 检查当前设备是否支持 CUDA (Nvidia GPU)，如果不行则回退到 CPU
    print(f"🔍 设备检测: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
    
    if torch.cuda.is_available():
        # 打印显卡型号，确认没有跑在核显或者错误的卡上
        print(f"GPU 型号: {torch.cuda.get_device_name()}")
        # 打印显存大小，确认是否有足够显存跑大模型 (1e9 = 1GB)
        print(f"显存大小: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

    # -----------------------------------------------------------------------------
    # 2. 随机种子设置
    # -----------------------------------------------------------------------------
    # 锁定随机种子 (Seed) 是复现实验结果的关键。
    # 它可以保证每次运行代码时，参数初始化和数据打乱的顺序都是一模一样的。
    set_seed(42)

    # -----------------------------------------------------------------------------
    # 3. 初始化模型配置
    # -----------------------------------------------------------------------------
    # 实例化配置类，这里定义了模型的所有超参数（层数、维度等）
    config = ModelConfig()
    print(f"\n📋 模型配置概览:")
    # d: 维度 (dimension), L: 层数 (layers), H: 头数 (heads), ff: 前馈网络维度
    print(f"   架构参数: {config.d_model}d, {config.n_layers}L, {config.n_heads}H, {config.d_ff}ff")
    print(f"   训练参数: 共 {config.max_steps} 步, Batch Size {config.batch_size}")
    print(f"   数据规模: 上限 {config.max_tokens:,} Token, 最大序列长度 {config.max_seq_len}")

    # -----------------------------------------------------------------------------
    # 4. 数据加载与预处理
    # -----------------------------------------------------------------------------
    # 加载文本数据并进行 Tokenization（分词）。通常这一步会把文本转成数字 ID 列表。
    texts, tokenizer, tokens = load_and_cache_data(config)
    
    # 封装成 PyTorch 的 Dataset 对象，负责按 max_seq_len 切割数据
    dataset = TextTokenDataset(tokens, config.max_seq_len)

    # -----------------------------------------------------------------------------
    # 5. 数据集划分 (Train/Val Split)
    # -----------------------------------------------------------------------------
    # 按照 9:1 的比例划分训练集和验证集
    # 验证集 (Val) 用来在训练中途测试模型是否过拟合
    val_size = len(dataset) // 10
    train_size = len(dataset) - val_size
    
    # 使用 random_split 进行随机切割，同样固定 generator 种子以保证可复现
    train_dataset, val_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42)
    )

    # 创建 DataLoader，负责按 Batch 批量输送数据
    # shuffle=True: 训练集必须打乱，防止模型记住数据的顺序
    # num_workers=2: 启用多进程加载数据。注意：在 Windows 上如果你遇到报错，可能需要改为 0
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, num_workers=0)

    print(f"📊 数据集分布: {len(train_dataset)} 训练样本, {len(val_dataset)} 验证样本")

    # -----------------------------------------------------------------------------
    # 6. 开始训练
    # -----------------------------------------------------------------------------
    # start_time = time.time()
    
    # # 调用核心训练函数，开始炼丹！
    # # 返回训练好的模型对象和最终的评估指标
    # model, final_metrics = train_model(config, train_loader, val_loader)
    
    # total_time = time.time() - start_time

    # # -----------------------------------------------------------------------------
    # # 7. 训练总结与汇报
    # # -----------------------------------------------------------------------------
    # print(f"\n🎉 训练圆满完成!")
    # print(f"⏱️ 总耗时: {total_time/60:.1f} 分钟")
    # print(f"🏆 最终成绩单:")
    # print(f"   验证损失 (Loss): {final_metrics['val_loss']:.4f} (越低越好)")
    # print(f"   验证准确率 (Acc): {final_metrics['val_accuracy']:.4f} (越高越好)")
    # # 困惑度 (Perplexity) 是衡量语言模型生成文本自然程度的核心指标
    # # 20 左右对于小模型来说是一个非常不错的成绩
    # print(f"   验证困惑度 (PPL): {final_metrics['val_perplexity']:.2f} (越低越好)")

🔍 设备检测: CUDA
GPU 型号: NVIDIA GeForce RTX 4080 Laptop GPU
显存大小: 12.9 GB
Set all seed to 42

📋 模型配置概览:
   架构参数: 384d, 6L, 8H, 1536ff
   训练参数: 共 500 步, Batch Size 24
   数据规模: 上限 500,000 Token, 最大序列长度 256
📦 正在从缓存中加载数据: data_cache/tokenized_data_2000_500000.pkl
✅ 加载成功：包含 2000 篇文档，共 500,000 个 Token
📊 数据集分布: 449770 训练样本, 49974 验证样本
